In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import math


def load_data(x_train_file, y_train_file, x_test_file, y_test_file):
    x_train = pd.read_csv(x_train_file).values
    y_train = pd.read_csv(y_train_file).values.flatten()
    x_test = pd.read_csv(x_test_file).values
    y_test = pd.read_csv(y_test_file).values.flatten()

    scaler = StandardScaler()
    x_train_scaled = scaler.fit_transform(x_train)
    x_test_scaled = scaler.transform(x_test)

    x_train_tensor = torch.tensor(x_train_scaled, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32)  # 目标值为一维 tensor
    x_test_tensor = torch.tensor(x_test_scaled, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32)  # 目标值为一维 tensor

    return x_train_tensor, y_train_tensor, x_test_tensor, y_test_tensor


class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(
            torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x):
        return x + self.pe[:, : x.size(1)]


class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, nhead):
        super(MultiHeadAttention, self).__init__()
        self.d_model = d_model
        self.nhead = nhead
        self.attn = nn.MultiheadAttention(d_model, nhead)

    def forward(self, query, key, value, mask=None):
        return self.attn(query, key, value, attn_mask=mask)[0]


class FeedForward(nn.Module):
    def __init__(self, d_model, dim_feedforward=2048):
        super(FeedForward, self).__init__()
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(dim_feedforward, d_model)

    def forward(self, x):
        return self.linear2(self.relu(self.linear1(x)))


class EncoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, nhead)
        self.ffn = FeedForward(d_model, dim_feedforward)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        src2 = self.self_attn(src, src, src)
        src = src + self.dropout(src2)
        src = self.norm1(src)
        src2 = self.ffn(src)
        src = src + self.dropout(src2)
        src = self.norm2(src)
        return src


class Encoder(nn.Module):
    def __init__(
        self, input_dim, d_model, nhead, num_layers, dim_feedforward=2048, dropout=0.1
    ):
        super(Encoder, self).__init__()
        self.embedding = nn.Linear(input_dim, d_model)
        self.positional_encoding = PositionalEncoding(d_model)
        self.layers = nn.ModuleList(
            [
                EncoderLayer(d_model, nhead, dim_feedforward, dropout)
                for _ in range(num_layers)
            ]
        )

    def forward(self, src):
        src = self.embedding(src)
        src = self.positional_encoding(src)
        for layer in self.layers:
            src = layer(src)
        return src


class TransformerModel(nn.Module):
    def __init__(
        self, input_dim, d_model, nhead, num_layers, dim_feedforward=2048, dropout=0.1
    ):
        super(TransformerModel, self).__init__()
        self.d_model = d_model
        self.encoder = Encoder(
            input_dim, d_model, nhead, num_layers, dim_feedforward, dropout
        )
        self.fc_out = nn.Linear(d_model, 1)

    def forward(self, src):
        memory = self.encoder(src)
        output = self.fc_out(memory.mean(dim=1))
        return output


def train_model(
    x_train, y_train, x_test, y_test, batch_size=32, epochs=50, learning_rate=0.001
):

    train_dataset = TensorDataset(x_train, y_train)
    val_dataset = TensorDataset(x_test, y_test)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    model = TransformerModel(
        input_dim=x_train.shape[1], d_model=10, nhead=8, num_layers=4
    )
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()

    train_losses = []
    val_losses = []
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            output = model(batch_X)
            loss = criterion(output, batch_y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        train_losses.append(total_loss / len(train_loader))

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch_X, batch_y in val_loader:
                output = model(batch_X)
                loss = criterion(output, batch_y)
                val_loss += loss.item()

        val_losses.append(val_loss / len(val_loader))
        print(
            f"Epoch {epoch+1}/{epochs}, Train Loss: {train_losses[-1]}, Validation Loss: {val_losses[-1]}"
        )

    plt.figure(figsize=(10, 6))
    plt.plot(range(epochs), train_losses, label="Training Loss")
    plt.plot(range(epochs), val_losses, label="Validation Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.title("Training and Validation Losses")
    plt.show()

    return model


def evaluate_model(model, x_test, y_test):
    model.eval()
    with torch.no_grad():
        predictions = model(x_test).numpy()
        true_values = y_test.numpy()

    plt.figure(figsize=(10, 6))
    plt.scatter(true_values, predictions, color="blue")
    plt.plot(
        [min(true_values), max(true_values)],
        [min(true_values), max(true_values)],
        color="red",
        linestyle="--",
    )
    plt.xlabel("True Values")
    plt.ylabel("Predictions")
    plt.title("True vs Predicted")
    plt.show()


x_train_file = "../set/x_train.csv"
y_train_file = "../set/y_train.csv"
x_test_file = "../set/x_test.csv"
y_test_file = "../set/y_test.csv"

x_train, y_train, x_test, y_test = load_data(
    x_train_file, y_train_file, x_test_file, y_test_file
)
d_model = 8
nhead = 8
model = TransformerModel(
    input_dim=x_train.shape[1], d_model=d_model, nhead=nhead, num_layers=4
)  
evaluate_model(model, x_test, y_test)

RuntimeError: The size of tensor a (18) must match the size of tensor b (8) at non-singleton dimension 1